In [12]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [13]:
from dotenv import load_dotenv
import os
load_dotenv() 

True

In [14]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [15]:
import src.config as config
from src.inference import (
    get_feature_store
)

from datetime import datetime, timezone, timedelta

def get_current_utc_hour_ceiled():
    """Get current UTC time ceiled to the next hour"""
    current = datetime.now(timezone.utc)
    if current.minute > 0 or current.second > 0:
        # Ceil to next hour
        current = current + timedelta(hours=1)

    # Reset minutes, seconds, and microseconds
    current = current.replace(minute=0, second=0, microsecond=0)
    return current

# Usage
current_hour = get_current_utc_hour_ceiled()
print(f"Current UTC hour (ceiled): {current_hour}")

Current UTC hour (ceiled): 2025-03-05 18:00:00+00:00


In [16]:
def fetch_predictions():
    current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.pickup_hour == current_hour)

    return query.read()

In [17]:
fs = get_feature_store()
fg = fs.get_feature_group(
    name=config.FEATURE_GROUP_MODEL_PREDICTION,
    version=1
)

query = fg.select_all()

2025-03-05 12:20:58,723 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 12:20:58,730 INFO: Initializing external client
2025-03-05 12:20:58,733 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 12:20:59,381 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215689


In [18]:
df= query.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.73s) 


In [19]:
df

,pickup_location_id,predicted_demand,pickup_hour
0,8,0.0,2025-03-05 04:00:00+00:00
1,88,14.0,2025-03-05 04:00:00+00:00
2,76,1.0,2025-03-05 04:00:00+00:00
3,31,0.0,2025-03-05 04:00:00+00:00
4,71,1.0,2025-03-05 04:00:00+00:00
...,...,...,...
3017,80,2.0,2025-03-05 18:00:00+00:00
3018,6,0.0,2025-03-05 18:00:00+00:00
3019,66,0.0,2025-03-05 18:00:00+00:00
3020,119,0.0,2025-03-05 18:00:00+00:00


In [22]:
from datetime import datetime, timezone, timedelta
import pandas as pd
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
    next_hour = pd.Timestamp(next_hour).tz_localize(None).tz_localize("UTC")



    # Extract components from the rounded-up time
    year = next_hour.year
    month = next_hour.month
    day = next_hour.day
    hour = next_hour.hour

    # Create date strings in YYYY-MM-DD format
    current_date = f"{year}-{month:02d}-{day:02d}"
    next_date = next_hour + timedelta(days=1)
    next_date_str = next_date.strftime("%Y-%m-%d")

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_MODEL_PREDICTION,
        version=1
    )

    # # First get the full day's data from Hopsworks
    # df = fg.filter(
    #     (fg.pickup_hour >= current_date) &
    #     (fg.pickup_hour < next_date_str)
    # ).read()

    # Then filter for next hour in the DataFrame
    hour_str = f"{current_date} {hour:02d}:00"
    df_hour = df[df['pickup_hour'] == hour_str]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Querying for date range: {current_date} to {next_date_str}")
    print(f"Filtering for hour: {hour_str}")
    print(f"Found {len(df_hour)} records")
    return df_hour

# Example usage:
predictions = fetch_next_hour_predictions()

2025-03-05 12:21:23,586 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 12:21:23,591 INFO: Initializing external client
2025-03-05 12:21:23,593 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 12:21:24,137 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215689
Current UTC time: 2025-03-05 17:21:23.573304+00:00
Next hour: 2025-03-05 18:00:00+00:00
Querying for date range: 2025-03-05 to 2025-03-06
Filtering for hour: 2025-03-05 18:00
Found 252 records


In [23]:
now = datetime.now(timezone.utc)

In [24]:
predictions

,pickup_location_id,predicted_demand,pickup_hour
2770,168,-0.0,2025-03-05 18:00:00+00:00
2771,197,0.0,2025-03-05 18:00:00+00:00
2772,169,1.0,2025-03-05 18:00:00+00:00
2773,196,0.0,2025-03-05 18:00:00+00:00
2774,209,9.0,2025-03-05 18:00:00+00:00
...,...,...,...
3017,80,2.0,2025-03-05 18:00:00+00:00
3018,6,0.0,2025-03-05 18:00:00+00:00
3019,66,0.0,2025-03-05 18:00:00+00:00
3020,119,0.0,2025-03-05 18:00:00+00:00


In [25]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3022 entries, 0 to 3021
Data columns (total 3 columns):
 #   Column              Non-Null Count  Dtype                  
---  ------              --------------  -----                  
 0   pickup_location_id  3022 non-null   int32                  
 1   predicted_demand    3022 non-null   float64                
 2   pickup_hour         3022 non-null   datetime64[us, Etc/UTC]
dtypes: datetime64[us, Etc/UTC](1), float64(1), int32(1)
memory usage: 59.2 KB


In [26]:
import pandas as pd
current_date = pd.Timestamp.now(tz='Etc/UTC').ceil('h')

In [27]:
dt = current_date.ceil('h')


In [28]:
fs = get_feature_store()  
fg = fs.get_feature_group(  
    name=config.FEATURE_GROUP_MODEL_PREDICTION,  
    version=1  
)  
query = fg.select_all() 
# query = query.filter(fg.pickup_hour >= "2025-02-13 04:00")
query = query.filter(fg.pickup_hour > "2025-02-13 05:00:00")  
results = query.read()

2025-03-05 12:21:40,431 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 12:21:40,436 INFO: Initializing external client
2025-03-05 12:21:40,436 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 12:21:41,087 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215689
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.95s) 


In [29]:
fg.filter((fg.pickup_hour >= "2025-02-13") & (fg.pickup_hour < "2025-02-14")).read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (10.22s) 


,pickup_location_id,predicted_demand,pickup_hour


In [30]:
results

,pickup_location_id,predicted_demand,pickup_hour
0,8,0.0,2025-03-05 04:00:00+00:00
1,88,14.0,2025-03-05 04:00:00+00:00
2,76,1.0,2025-03-05 04:00:00+00:00
3,31,0.0,2025-03-05 04:00:00+00:00
4,71,1.0,2025-03-05 04:00:00+00:00
...,...,...,...
3017,80,2.0,2025-03-05 18:00:00+00:00
3018,6,0.0,2025-03-05 18:00:00+00:00
3019,66,0.0,2025-03-05 18:00:00+00:00
3020,119,0.0,2025-03-05 18:00:00+00:00


In [31]:
def fetch_next_hour_predictions():
    # Get current UTC time and round up to next hour
    now = datetime.now(timezone.utc)
    next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)

    fs = get_feature_store()
    fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)

    # Then filter for next hour in the DataFrame
    df_hour = df[df["pickup_hour"] == next_hour]

    print(f"Current UTC time: {now}")
    print(f"Next hour: {next_hour}")
    print(f"Found {len(df_hour)} records")
    return df_hour

In [32]:
predictions = fetch_next_hour_predictions()

2025-03-05 12:22:00,007 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 12:22:00,029 INFO: Initializing external client
2025-03-05 12:22:00,031 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 12:22:00,777 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215689
Current UTC time: 2025-03-05 17:22:00.007048+00:00
Next hour: 2025-03-05 18:00:00+00:00
Found 252 records


In [33]:
fs = get_feature_store()
fg = fs.get_feature_group(name=config.FEATURE_GROUP_MODEL_PREDICTION, version=1)



2025-03-05 12:22:01,643 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-05 12:22:01,649 INFO: Initializing external client
2025-03-05 12:22:01,651 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-05 12:22:02,261 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1215689


In [34]:
df = fg.read()

Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.77s) 


In [35]:
# Convert pickup_hour to datetime with UTC timezone
df["pickup_hour"] = pd.to_datetime(df["pickup_hour"]).dt.tz_convert("UTC")

# Debugging: Check unique timestamps
print("Unique pickup_hour values in df:")
print(df["pickup_hour"].unique())  


Unique pickup_hour values in df:
<DatetimeArray>
['2025-03-05 04:00:00+00:00', '2025-03-05 08:00:00+00:00',
 '2025-03-05 09:00:00+00:00', '2025-03-05 10:00:00+00:00',
 '2025-03-05 11:00:00+00:00', '2025-03-05 12:00:00+00:00',
 '2025-03-05 13:00:00+00:00', '2025-03-05 14:00:00+00:00',
 '2025-03-05 15:00:00+00:00', '2025-03-05 16:00:00+00:00',
 '2025-03-05 17:00:00+00:00', '2025-03-05 18:00:00+00:00']
Length: 12, dtype: datetime64[us, UTC]


In [36]:
df["pickup_hour"].max()

Timestamp('2025-03-05 18:00:00+0000', tz='UTC')

In [37]:
now = datetime.now(timezone.utc)
next_hour = (now + timedelta(hours=1)).replace(minute=0, second=0, microsecond=0)
print(next_hour)

2025-03-05 18:00:00+00:00


In [38]:
df[df["pickup_hour"] == next_hour]

,pickup_location_id,predicted_demand,pickup_hour
2770,168,-0.0,2025-03-05 18:00:00+00:00
2771,197,0.0,2025-03-05 18:00:00+00:00
2772,169,1.0,2025-03-05 18:00:00+00:00
2773,196,0.0,2025-03-05 18:00:00+00:00
2774,209,9.0,2025-03-05 18:00:00+00:00
...,...,...,...
3017,80,2.0,2025-03-05 18:00:00+00:00
3018,6,0.0,2025-03-05 18:00:00+00:00
3019,66,0.0,2025-03-05 18:00:00+00:00
3020,119,0.0,2025-03-05 18:00:00+00:00
